In [37]:
import pandas as pd
from preprocess import get_data

In [38]:
def pre_series(series, countries):
    process_series = series.groupby('Country/Region').sum().drop(['Lat', 'Long'], axis=1).loc[countries, :].T[
    countries].fillna(0)
    process_series.index = pd.to_datetime(process_series.index)
    return process_series

In [39]:
indicator_year = 2019
indicators = pd.read_excel('raw_data/indicators_all_countries.xlsx')[
    ['Country', 'Indicator', 'Unit', indicator_year]]
# policies = pd.read_excel('raw_data/policies.xlsx')

In [40]:
# 'confirmed', 'deaths', 'recovered'
confirmed = pd.read_csv('raw_data/COVID/time_series_covid19_' + 'confirmed' + '_global.csv')
deaths = pd.read_csv('raw_data/COVID/time_series_covid19_' + 'deaths' + '_global.csv')
recovered = pd.read_csv('raw_data/COVID/time_series_covid19_' + 'recovered' + '_global.csv')

In [41]:
countries = sorted(list((set(confirmed['Country/Region'])).intersection(set(indicators['Country'])))) # set(policies.entity).intersection

In [42]:
confirmed = pre_series(confirmed, countries)
deaths = pre_series(deaths, countries)
recovered = pre_series(recovered, countries)

In [68]:
# mortality/recovery rate 死亡率恢复率
mortality = (deaths / confirmed).dropna(how='all')
recovery = (recovered / confirmed).dropna(how='all')

In [70]:
def indicator_influence(indicators, rates):
    """
    Perason correlation
    """
    indicators_values = indicators.pivot_table(index=['Indicator', 'Unit'], columns='Country', values=indicator_year)[countries].fillna(0).T
    indicator_mortality = {}
    for i in range(len(indicators_values.columns)):
        indicator_mortality[indicators_values.columns[i]] = (indicators_values.iloc[:,i].corr(rates.iloc[-1]))
    indicator_mortality = sorted(indicator_mortality.items(), key=lambda x:-abs(x[1]))
    rate = []
    for x in indicator_mortality:
        rate.append([x[0][0], x[0][1], x[1]])
    return pd.DataFrame(rate, columns=['Indicator', 'Unit', 'influence'])

In [72]:
indicator_influence(indicators, mortality).to_csv('output/step_two/indicator_influence/indicator_mortality.csv', index=False)

In [63]:
indicator_influence(indicators, recovery).to_csv('output/step_two/indicator_influence/indicator_recovery.csv', index=False)

In [29]:
len(countries)

174